In [12]:
import datetime

def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )
            
yesterday = datetime.date.today() - datetime.timedelta(days = 1)
start_date = datetime.date( year = 2018, month = 2, day = 1 )

my_dates = []

for date in daterange( start_date, yesterday ):
    my_dates.append(date.strftime('%a-%d-%m-%y'))

print(my_dates[0])

Thu-01-02-18


In [13]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.prosoccer.eu/football/'

#date = my_dates[0]

for date in my_dates:

    my_url = base_url + date
    page = urllib.request.urlopen(my_url)
    soup = BeautifulSoup(page)
    right_div=soup.find_all('div', class_='table-row')
    A=[]
    B=[]
    C=[]
    D=[]
    E=[]
    F=[]
    G=[]
    H=[]
    I=[]
    J=[]
    K=[]
    L=[]
    M=[]
    N=[]
    O=[]
    P=[]
    Q=[]

    for row in right_div:
        cells = row.findAll('div')
        if len(cells)==17:
            A.append(cells[0].find(text=True))
            B.append(cells[2].find(text=True))
            C.append(cells[3].find(text=True))
            D.append(cells[4].find(text=True))
            E.append(cells[5].find(text=True))
            F.append(cells[6].find(text=True))
            G.append(cells[7].find(text=True))
            H.append(cells[8].find(text=True))
            I.append(cells[9].find(text=True))
            J.append(cells[10].find(text=True))
            K.append(cells[11].find(text=True))
            L.append(cells[12].find(text=True))
            M.append(cells[13].find(text=True))
            N.append(cells[14].find(text=True))
            O.append(cells[15].find(text=True))
            P.append(cells[16].find(text=True))
            
    df=pd.DataFrame(A,columns=['Time'])
    df['Comp']=B
    df['Home']=C
    df['Away']=D
    df['Tip']=E
    df['H']=F
    df['D']=G
    df['A']=H
    df['1']=I
    df['X']=J
    df['2']=K
    df['U']=L
    df['O']=M
    df['U2.5']=N
    df['O2.5']=O
    df['FT']=P
    
    file_name = 'csv/' + date + '.csv'
    df.to_csv(file_name, encoding='utf-8', index=False)


/home/groninge/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/groninge/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [14]:
import pandas as pd

# cat `ls -1tr` >> ../big.csv

all = pd.read_csv('big.csv')

all.head()

,Time,Comp,Home,Away,Tip,H,D,A,1,X,2,U,O,U2.5,O2.5,FT
0,00:00,MX2,Mineros de Zacatecas,Juárez,X1,53,23,24,2.14,3.20,3.20,42,58,1.57,2.35,4 - 2
1,04:00,MX2,Dorados,Alebrijes De Oaxaca,NaN,0,0,0,,,,0,0,,,1 - 2
2,05:00,MX2,Dorados,Alebrijes de Oaxaca,X2,29,32,38,2.62,3.20,2.50,72,28,1.54,2.30,1 - 2
3,21:45,WAL1,Connah's Quay,Bala Town,1,62,18,19,,,,26,74,,,1 - 1
4,Time,Comp,Home,Away,Tip,H,D,A,1,X,2,U,O,U2.5,O2.5,FT


In [4]:
all2.get_dtype_counts()


object    16
dtype: int64

In [36]:
import numpy as np

all2 = all.copy()

all2 = all2.dropna() # drop all rows with NaN
all2 = all2.query('Time != "Time"') # drop all rows with the labels in them (from catting csv's)
all2 = all2.replace(regex={r'X1':'1X', '2X':'X2'}) # make sure all Tips are the same
all2[['HG','AG']] = all2['FT'].str.split(' - ',expand=True) # split FT score in Home Goals and Away Goals
all2= all2.drop('FT', axis=1) # drop FT column

conditions = [
    (all2['HG'] > all2['AG']),
    (all2['HG'] == all2['AG']),
    (all2['HG'] < all2['AG'])]
choices = ['1', 'X', '2']

all2['FT'] = np.select(conditions, choices)

all2

all2.to_csv('clean_data.csv', encoding='utf-8', index=False)


In [76]:
df = pd.read_csv('clean_data.csv')

df.drop(['Time', 'Comp', 'Home', 'Away', 'Tip', '1', 'X', '2', 'U2.5', 'O2.5','HG', 'AG'],1, inplace=True)

df.FT = df.FT.astype('str')

df.head()




,H,D,A,U,O,FT
0,53,23,24,42,58,1
1,29,32,38,72,28,2
2,62,18,19,26,74,X
3,81,12,6,32,68,1
4,63,28,9,77,23,1


In [77]:
# Separate into feature set and target variable
#FT = Full Time Result (1=Home Win, X=Draw, 2=Away Win)
X_all = df.drop(['FT'],1)
y_all = df['FT']

# Standardising the data.
from sklearn.preprocessing import scale


cols = [['H', 'D' , 'A', 'U', 'O']]
for col in cols:
    X_all[col] = scale(X_all[col])

In [78]:
from sklearn.cross_validation import train_test_split

# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 50,
                                                    random_state = 2,
                                                    stratify = y_all)

In [79]:
#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
#print("Processed feature columns ({} total features):\n{}").format(len(X_all.columns), list(X_all.columns))

In [80]:
X_all

,H,D,A,U,O
0,0.465970,-0.325292,-0.374452,-0.564631,0.564631
1,-0.868939,0.954442,0.496457,0.992331,-0.992331
2,0.966561,-1.036256,-0.685492,-1.395010,1.395010
3,2.023364,-1.889412,-1.494194,-1.083618,1.083618
4,1.022182,0.385671,-1.307570,1.251824,-1.251824
5,-1.425151,3.656104,-0.001205,2.186001,-2.186001
6,-0.813318,0.101286,0.931912,-0.149441,0.149441
7,0.966561,0.101286,-1.120947,0.784736,-0.784736
8,0.465970,0.101286,-0.561076,0.317647,-0.317647
9,1.745258,-1.462834,-1.307570,-1.083618,1.083618


In [90]:
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC

#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    #print("Trained model in {:.4f} seconds").format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    #print("Made predictions in {:.4f} seconds.").format(end - start)    
    
    return f1_score(target, y_pred, pos_label='1', average='micro'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print("Training a {} using a training set size of {}. . .").format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    #print("F1 score and accuracy score for training set: {0:.4f} , {1:.4f}.").format(f1 , acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print(f1, acc)
    #print("F1 score and accuracy score for test set: {0:.4f} , {1:.4f}.").format(f1 , acc)


In [91]:
# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print('')

/home/groninge/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1045: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


0.47694001100715466 0.47694001100715466
0.48 0.48

0.48046230049532196 0.48046230049532196
0.46 0.46

0.48970831040176116 0.48970831040176116
0.41999999999999993 0.42



/home/groninge/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/groninge/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1045: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/groninge/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
